In [25]:
import pandas as pd
import numpy as np
import json

In [4]:
df = pd.read_csv('../data/raw/dataset.csv')

C:\Users\jessi\AppData\Local\Temp\ipykernel_91692\2880239794.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/dataset.csv')


Dropping the 'weight' column since the number of missing values is too high and filling some NaN values across the database

In [ ]:
# Drop the weight column due to high missing value percentage
df = df.drop(columns=['weight'])

# Drop the lines where diag_1 is missing, since the percentage of missing values is low
df = df.dropna(subset=['diag_1'])

# Fill missing values in diag_2 and diag_3 with 'Unknown'
df['diag_2'] = df['diag_2'].fillna('Unknown')
df['diag_3'] = df['diag_3'].fillna('Unknown')

# Fill missing values in medical_specialty with 'Unknown'
df['medical_specialty'] = df['medical_specialty'].fillna('Unknown')

# Fill missing values in payer_code with 'Missing'
df['payer_code'] = df['payer_code'].fillna('Missing')

# Fill missing values in race with 'Unknown'
df['race'] = df['race'].fillna('Unknown')

# Fill missing values in max_glu_serum with 'Not_measured'
df['max_glu_serum'] = df['max_glu_serum'].fillna('Not_measured')

# Fill missing values in A1Cresult with 'Not_measured'
df['A1Cresult'] = df['A1Cresult'].fillna('Not_measured')

In [7]:
df

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,Missing,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,Missing,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,Missing,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,Missing,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,Missing,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),1,3,7,3,MC,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),1,4,5,5,MC,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),1,1,7,1,MC,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),2,3,7,10,MC,...,No,Up,No,No,No,No,No,Ch,Yes,NO


For the diagnose columns, we are going to reduce the number of values by getting the ICD9 Diagnoses ranges

In [12]:
df['diag_1'].info()

<class 'pandas.core.series.Series'>
Index: 101745 entries, 0 to 101765
Series name: diag_1
Non-Null Count   Dtype 
--------------   ----- 
101745 non-null  object
dtypes: object(1)
memory usage: 1.6+ MB


In [15]:
def read_json(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data

In [18]:
data = read_json('../data/external/ICD9_Diagnosis.json')
data.items()

dict_items([('001,139', {'description': 'Infectious And Parasitic Diseases', 'short': 'InfectiousDiseases'}), ('140,239', {'description': 'Neoplasms', 'short': 'Neoplasms'}), ('240,279', {'description': 'Endocrine, Nutritional And Metabolic Diseases, And Immunity Disorders', 'short': 'EndocrineDiseases'}), ('280,289', {'description': 'Diseases Of The Blood And Blood-Forming Organs', 'short': 'BloodDiseases'}), ('290,319', {'description': 'Mental Disorders', 'short': 'MentalDisorders'}), ('320,389', {'description': 'Diseases Of The Nervous System And Sense Organs', 'short': 'NervousSystemDiseases'}), ('390,459', {'description': 'Diseases Of The Circulatory System', 'short': 'CirculatorySystemDiseases'}), ('460,519', {'description': 'Diseases Of The Respiratory System', 'short': 'RespiratorySystemDiseases'}), ('520,579', {'description': 'Diseases Of The Digestive System', 'short': 'DigestiveSystemDiseases'}), ('580,629', {'description': 'Diseases Of The Genitourinary System', 'short': 'G

In [33]:
def get_icd9_range(code: str):
    icd9_data = read_json('../data/external/ICD9_Diagnosis.json')

    if code is np.nan or code == 'Unknown':
        code_type = 'Unknown'
    elif code.startswith('E'):
        code_type = icd9_data["E"]["short"]
    elif code.startswith('V'):
        code_type = icd9_data["V"]["short"]
    else:
        numeric_ranges = icd9_data["numeric"]
        code = code.split('.')[0]
        for code_range,info in numeric_ranges.items():
            range_bounds = code_range.split(',')
            if int(range_bounds[0]) <= int(code) <= int(range_bounds[1]):
                code_type = info['short']

    return code_type

In [36]:
df['diag_1_range'] = df['diag_1'].apply(lambda x: get_icd9_range(x))
df['diag_2_range'] = df['diag_2'].apply(lambda x: get_icd9_range(x))
df['diag_3_range'] = df['diag_3'].apply(lambda x: get_icd9_range(x))

In [37]:
print("Number of unique values before: ")
print(f"diag_1: {df['diag_1'].nunique()}")
print(f"diag_2: {df['diag_2'].nunique()}")  
print(f"diag_3: {df['diag_3'].nunique()}")

print("\nNumber of unique values after: ")
print(f"diag_1_range: {df['diag_1_range'].nunique()}")
print(f"diag_2_range: {df['diag_2_range'].nunique()}")
print(f"diag_3_range: {df['diag_3_range'].nunique()}")

Number of unique values before: 
diag_1: 716
diag_2: 749
diag_3: 790

Number of unique values after: 
diag_1_range: 18
diag_2_range: 19
diag_3_range: 19
